In [31]:
import pandas as pd
import os
import re

In [33]:
df = pd.DataFrame()
#os.chdir("../data/alternatives_corpora")
for filename in os.listdir():
    if ".txt" not in filename:
        continue
    alternatives = {'a':[],'b':[],'c':[],'d':[]}
    with open(filename) as f:
        for line in f.readlines():
            if line.strip():
                split = re.split("[a-z]\s[A-Z]{3,4}\s\d*",line.strip(),1)
                if len(split)>1:
                    alt = line.strip()[0]
                    alternatives[alt].append(split[1])
    to_append = {
        "date":filename.split(".txt")[0],
    }
    for alternative in alternatives.keys():
        to_append["alt {} corpus".format(alternative)] = "\n".join(alternatives[alternative])
    df = df.append(to_append,ignore_index=True)
alt_corpus_df = df.sort_values(by="date").reset_index(drop=True)

In [34]:
label_df = pd.read_csv("../../output/fed_targets_with_alternatives.csv")

In [35]:
print(f'Length of alternative corpus df: {len(alt_corpus_df)}')
print(f'Length of label_df: {len(label_df)}')
for col in label_df.columns:
    print(col)

Length of alternative corpus df: 424
Length of label_df: 169
Unnamed: 0
date
ffrtarget
target_before
target_after
decision
alt_a_rate
alt_b_rate
alt_c_rate
alt_d_rate
alt_e_rate
bluebook_treatment_size_alt_a
bluebook_treatment_size_alt_b
bluebook_treatment_size_alt_c
bluebook_treatment_size_alt_d
bluebook_treatment_size_alt_e


In [36]:
label_df = label_df[
    [
        "date",
        "decision",
        "bluebook_treatment_size_alt_a",
"bluebook_treatment_size_alt_b",
"bluebook_treatment_size_alt_c",
"bluebook_treatment_size_alt_d",
"bluebook_treatment_size_alt_e"
    ]
]

In [37]:
label_df['date'] = pd.to_datetime(label_df['date'])
alt_corpus_df['date'] = pd.to_datetime(alt_corpus_df['date'])

In [38]:
label_df = label_df[(label_df.date.dt.year>1987)&(label_df.date.dt.year<2009)]
alt_corpus_df = alt_corpus_df[(alt_corpus_df.date.dt.year>1987)&(alt_corpus_df.date.dt.year<2009)]

label_df['month'] = label_df.date.dt.month
label_df['year'] = label_df.date.dt.year

alt_corpus_df['month'] = alt_corpus_df.date.dt.month
alt_corpus_df['year'] = alt_corpus_df.date.dt.year


print(len(alt_corpus_df))
print(len(label_df))
#print(pd.concat([label_df[['month','year']],alt_corpus_df[['month','year']]]).drop_duplicates(keep=False))


168
169


In [61]:

#REMOVE DUPLICATE DATE ON SEPTEMBER 15th 2003
#print(len(label_df))
repeat = label_df[label_df[['month','year']].duplicated()]
#print(repeat)
label_df = label_df[label_df.date!=pd.to_datetime("2003-09-15")]
#print(label_df[(label_df.date.dt.year==2003)&(label_df.date.dt.month==9)])

indices = (pd.concat([label_df[['month','year']],alt_corpus_df[['month','year']]]).drop_duplicates(keep=False))

print(indices)
print("="*50)
label_indices = indices[0:len(indices)//2]
alt_indices = indices[len(indices)//2:]
#print(label_indices)
#print(alt_indices)

#FIX DATES WHICH APPEAR AT END OF MONTH
#date_revisions = {'1992-06-30':'1992-07-01','1995-01-31':'1995-02-01','1998-06-30':'1998-07-01'}
#print(list(date_revisions.keys())[0])
#print(alt_corpus_df[ alt_corpus_df.date == list(date_revisions.keys())[0]])


for label_index, alt_index in zip(label_indices.index,alt_indices.index):
    print(f'Label Index:{label_index}')
    print(f'Alt Index:{alt_index}')
    #print(alt_corpus_df.loc[alt_index,"date"])
    #print(label_df.loc[label_index,"date"])
    alt_corpus_df.loc[alt_index,"date"] = label_df.loc[label_index,"date"]



     month  year
35       7  1992
56       2  1995
83       7  1998
280      6  1992
301      1  1995
328      6  1998
Label Index:35
Alt Index:280
Label Index:56
Alt Index:301
Label Index:83
Alt Index:328
     month  year
35       7  1992
56       2  1995
83       7  1998
280      6  1992
301      1  1995
328      6  1998


In [62]:
label_df['month'] = label_df.date.dt.month
label_df['year'] = label_df.date.dt.year

alt_corpus_df['month'] = alt_corpus_df.date.dt.month
alt_corpus_df['year'] = alt_corpus_df.date.dt.year
print(len(alt_corpus_df))
print(len(label_df))
print(pd.concat([label_df[['month','year']],alt_corpus_df[['month','year']]]).drop_duplicates(keep=False))


168
168
Empty DataFrame
Columns: [month, year]
Index: []


In [63]:

alt_corpus_df = alt_corpus_df[[x for x in alt_corpus_df.columns if x!="date"]]
merge_df = pd.merge(label_df,alt_corpus_df,on=['month','year'])


#print(alt_corpus_df)



In [64]:
merge_df = merge_df[['date', 'alt a corpus','bluebook_treatment_size_alt_a',
                     'alt b corpus','bluebook_treatment_size_alt_b',
                     'alt c corpus','bluebook_treatment_size_alt_c',
                     'alt d corpus','bluebook_treatment_size_alt_d',
                     'decision'
                    ]]

In [65]:
print(len(merge_df))

168


In [66]:
merge_df.to_csv("../../output/alternative_outcomes_and_corpus.csv")

          date                                       alt a corpus  \
0   1988-02-10   the more rapid growth associated with the alt...   
1   1988-03-29  -- under alternative a , federal funds would b...   
2   1988-05-17   such a measure on the heels of the two recent...   
3   1988-06-30   this might involve a judgment that there was ...   
4   1988-08-16                                                      
5   1988-09-20   under alternative a , m2 and m3 would strengt...   
6   1988-11-01   under alternative a , money market interest r...   
7   1988-12-14                                                      
8   1989-02-08  -- the ranges of alternative a are those adopt...   
9   1989-03-28   with the lower money market rates of alternat...   
10  1989-05-16   m2 under all the alternatives embodies some i...   
11  1989-07-06   alternative\n i gives the most scope for even...   
12  1989-08-22  -- under alternative a , seasonal credit would...   
13  1989-10-03   under alternative